In [1]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
import math
from numpy import random
from IPython.display import HTML
import torchvision.models as torch_models
from base64 import b64encode
import os
from IPython.display import Video
from utils import *
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from cls_detection import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(device)

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225]),
])

yolo_model_path = "weights/detect_large_v2.pt"
model = YOLO(yolo_model_path)
classNames = ['basketball', 'hoop', 'made', 'person']

cuda


In [2]:
modelPath = "cls_chkpoint_resnet18/checkpoint_2024-01-01-12-56_lr_0.003_batch_128/checkpoint_2024-01-01-12-56_lr_0.003_batch_128/best_model.pth"
batch_size = 128 
conf = 0.3

cls_model = load_resnet18(modelPath)
#resnet50 = load_resnet50(resnet50_path)

In [ ]:
timestamps_all_made = inference_video(model = model,
                                      cls_model = cls_model,
                                      video_path = "video_test_dataset/all_made.mp4",
                                      save_result_vid = True,
                                      display_result = True,
                                      show_progress=True,
                                      cls_conf_threshold=0.3,
                                      device="cuda",
                                      show_score_prob=True,
                                      detect_conf_threshold=0.7,
                                      batch_mode=True,
                                      batch_size=128,
                                      ffmpeg_path="ffmpeg",
                                      num_buffer_frames=15
                                      )
print(len(timestamps_all_made))

In [9]:
inference_video(model = model,
                cls_model = cls_model,
                video_path = "video_test_dataset/1/3.mp4",
                save_result_vid = True,
                display_result = True,
                show_progress=True,
                cls_conf_threshold=0.3,
                device="cuda",
                show_score_prob=True,
                detect_conf_threshold=0.7,
                batch_mode=True,
                batch_size=128,
                ffmpeg_path="ffmpeg"
                )

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


[(1.7, array([    0.42897], dtype=float32))]

In [85]:
timestamps_all_made

[(1.07, array([    0.54381], dtype=float32)),
 (5.21, array([    0.57434], dtype=float32)),
 (9.58, array([    0.98205], dtype=float32)),
 (13.08, array([    0.56987], dtype=float32)),
 (15.12, array([     0.9445], dtype=float32)),
 (18.85, array([    0.94922], dtype=float32)),
 (20.89, array([    0.42494], dtype=float32)),
 (23.52, array([    0.46399], dtype=float32)),
 (25.66, array([    0.42818], dtype=float32)),
 (28.63, array([    0.82246], dtype=float32)),
 (30.66, array([    0.44506], dtype=float32)),
 (33.07, array([    0.42596], dtype=float32)),
 (35.1, array([    0.86683], dtype=float32)),
 (37.14, array([    0.57714], dtype=float32)),
 (39.31, array([    0.43201], dtype=float32)),
 (42.04, array([    0.75244], dtype=float32)),
 (44.08, array([    0.49582], dtype=float32)),
 (46.15, array([    0.40713], dtype=float32)),
 (63.9, array([    0.91305], dtype=float32)),
 (65.93, array([    0.60409], dtype=float32)),
 (69.2, array([    0.96218], dtype=float32)),
 (72.37, array([   